In [35]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import random

In [36]:
from sklearn.datasets import load_iris
# Loading Iris data
iris = load_iris()
X, y = iris.data, iris.target

# Data preprocessing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X,X_test,y,y_test=train_test_split(X_scaled,y,test_size=0.2,random_state=42)

In [37]:
# activation Function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
  exp_scores = np.exp(x - np.max(x)) # Shift values for numerical stability
  softmax_probs = exp_scores / np.sum(exp_scores, axis=0)
  return softmax_probs

In [38]:
# Lost function
def mse_loss(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

def one_hot_encoding(indices):
    num_classes = len(np.unique(indices))
    return np.eye(num_classes)[indices]

In [39]:
# Architecture of Neural Network 
input_size = X.shape[1]
hidden_size = 64
output_size = 3

epochs = 100
y_true_one_hot = one_hot_encoding(y)
y_test_one_hot = one_hot_encoding(y_test)


In [40]:
def forward_pass(X,weights_input_hidden,weights_hidden_output,bias_hidden,bias_output):
   # Forward pass
    hidden_layer_input = np.dot(X, weights_input_hidden)+bias_hidden
    hidden_layer_output = sigmoid(hidden_layer_input)
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output)+bias_output
    predicted_output = softmax(output_layer_input)
    return predicted_output

In [41]:


class HorseNet:
    def __init__(self, input_size, hidden_size, output_size, age='α'):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.age = age
        self.weights_input_hidden = np.random.uniform(0, 1, size=(input_size, hidden_size))
        self.weights_hidden_output = np.random.uniform(0, 1, size=(hidden_size, output_size))
        self.b1 = np.random.randn(hidden_size)
        self.b2 = np.random.randn(output_size)

    def get_weights(self):
        return self.weights_input_hidden, self.weights_hidden_output, self.b1, self.b2

class Population:
    def __init__(self, nb_horses):
        self.nb_horses = nb_horses

    def create_population(self, X, y_one_hot, input_size=4, hidden_size=128, output_size=3):
        horses = {}
        performances = []

        for i in range(self.nb_horses):
            horse_net = HorseNet(input_size, hidden_size, output_size, age=0)  # Creating a HorseNet instance with age 0
            weights_input_hidden, weights_hidden_output, bias_hidden, bias_output = horse_net.get_weights()
            horses[i] = weights_input_hidden, weights_hidden_output, bias_hidden, bias_output, horse_net.age

        for horse_id, (weights_input_hidden, weights_hidden_output, bias_hidden, bias_output, age) in horses.items():
            predicted_output = forward_pass(X, weights_input_hidden, weights_hidden_output, bias_hidden, bias_output)
            loss = mse_loss(y_one_hot, predicted_output)
            performances.append(loss)

        sorted_indices = np.argsort(performances)
        alpha_count = int(self.nb_horses * 0.1)
        beta_count = int(self.nb_horses * 0.2)
        gamma_count = int(self.nb_horses * 0.3)

        alpha_indices = sorted_indices[:alpha_count]
        beta_indices = sorted_indices[alpha_count:alpha_count + beta_count]
        gamma_indices = sorted_indices[alpha_count + beta_count:alpha_count + beta_count + gamma_count]
        delta_indices = sorted_indices[alpha_count + beta_count + gamma_count:]

        horses_ = {}
        for horse_id, (weights_input_hidden, weights_hidden_output, bias_hidden, bias_output, age) in horses.items():
            if horse_id in alpha_indices:
                horses_[horse_id] = weights_input_hidden, weights_hidden_output, bias_hidden, bias_output, 'α'
            if horse_id in beta_indices:
                horses_[horse_id] = weights_input_hidden, weights_hidden_output, bias_hidden, bias_output, 'β'
            if horse_id in gamma_indices:
                horses_[horse_id] = weights_input_hidden, weights_hidden_output, bias_hidden, bias_output, 'γ'
            if horse_id in delta_indices:
                horses_[horse_id] = weights_input_hidden, weights_hidden_output, bias_hidden, bias_output, 'δ'

        return horses_


# Create an instance of Population
population = Population(nb_horses=100)

# Create the population
horses = population.create_population(X,y_true_one_hot,input_size=input_size,hidden_size=hidden_size,output_size=output_size)






In [42]:
def grazing(weights, g_factor=1.5, u_bar=1.05, l_bar=0.95,wg=0.95):
    random_matrix= np.random.rand(*np.array(weights.shape))
    grazing_matrix= g_factor * (u_bar + random_matrix*l_bar)*weights
    g_factor=g_factor*wg
    return grazing_matrix,g_factor

In [43]:
def hierarchy(id_horse,horses,X,y_one_hot,wh=0.95,h_beta=0.9,h_gamma=0.5):
  performances={}
  for horse_id, (weights_input_hidden, weights_hidden_output,bias_hidden,bias_output,age_category) in horses.items():
        predicted_output = forward_pass(X, weights_input_hidden, weights_hidden_output,bias_hidden,bias_output)
        loss = mse_loss(y_one_hot, predicted_output)
        performances[horse_id]=loss

  best_horse_id=min(performances, key=performances.get)
  weights_input_hidden_best, weights_hidden_output_best,bias_hidden_best,bias_output_best, age_best = horses[best_horse_id]

  hierarchy_matrix_in=np.zeros_like(  weights_input_hidden_best)
  hierarchy_matrix_out=np.zeros_like(weights_hidden_output_best)
  hierarchy_matrix_bias_in=np.zeros_like(bias_hidden_best)
  hierarchy_matrix_bias_out=np.zeros_like(bias_output_best)


  weights_input_hidden, weights_hidden_output,bias_hidden,bias_output, age_category=horses[id_horse]
  if age_category=='β':
    hierarchy_matrix_in=h_beta*(weights_input_hidden_best-weights_input_hidden)
    hierarchy_matrix_out=h_beta*(weights_hidden_output_best-weights_hidden_output)
    hierarchy_matrix_bias_in=h_beta*(bias_hidden_best-bias_hidden)
    hierarchy_matrix_bias_out=h_beta*(bias_output_best-bias_output)
    h_beta=h_beta*wh
  if age_category=='γ':
    hierarchy_matrix_in=h_gamma*(weights_input_hidden_best-weights_input_hidden)
    hierarchy_matrix_out=h_gamma*(weights_hidden_output_best-weights_hidden_output)
    hierarchy_matrix_bias_in=h_gamma*(bias_hidden_best-bias_hidden)
    hierarchy_matrix_bias_out=h_gamma*(bias_output_best-bias_output)
    h_gamma=h_gamma*wh
  return hierarchy_matrix_in,hierarchy_matrix_out,hierarchy_matrix_bias_in,hierarchy_matrix_bias_out,h_beta,h_gamma

In [44]:
def sociability(id_horse,horses,s_beta=0.2,s_gamma=0.1,ws=0.95):
  N=len(horses)
  sociability_matrix_in=np.zeros_like(horses[0][0])
  sociability_matrix_out=np.zeros_like(horses[0][1])
  sociability_matrix_bias_in=np.zeros_like(horses[0][2])
  sociability_matrix_bias_out=np.zeros_like(horses[0][3])
  sum_weight_in=np.zeros_like(horses[0][0])
  sum_weight_out=np.zeros_like(horses[0][1])
  sum_weight_bias_in=np.zeros_like(horses[0][2])
  sum_weight_bias_out=np.zeros_like(horses[0][3])
  for horse_id, (weights_input_hidden, weights_hidden_output,bias_hidden,bias_output, age_category) in horses.items():
    sum_weight_in+=weights_input_hidden
    sum_weight_out+=weights_hidden_output
    sum_weight_bias_in+=bias_hidden
    sum_weight_bias_out+=bias_output

  weights_input_hidden, weights_hidden_output,bias_hidden,bias_output, age_category=horses[id_horse]
  if age_category=='β':
    sociability_matrix_in=s_beta*((sum_weight_in/N)-weights_input_hidden)
    sociability_matrix_out=s_beta*((sum_weight_out/N)-weights_hidden_output)

    sociability_matrix_bias_in=s_beta*((sum_weight_bias_in/N)-bias_hidden)
    sociability_matrix_bias_out=s_beta*((sum_weight_bias_out/N)-bias_output)
    s_beta=s_beta*ws
  if age_category=='γ':
    sociability_matrix_in=s_gamma*((sum_weight_in/N)-weights_input_hidden)
    sociability_matrix_out=s_gamma*((sum_weight_out/N)-weights_hidden_output)
    sociability_matrix_bias_in=s_gamma*((sum_weight_bias_in/N)-bias_hidden)
    sociability_matrix_bias_out=s_gamma*((sum_weight_bias_out/N)-bias_output)
    s_gamma=s_gamma*ws
  return sociability_matrix_in,sociability_matrix_out,sociability_matrix_bias_in,sociability_matrix_bias_out,s_beta,s_gamma

In [45]:
def imitation(id_horse,horses,X,y_one_hot,i_gamma=0.3,wi=0.95):
  pN=10
  performances={}
  for horse_id, (weights_input_hidden, weights_hidden_output,bias_hidden,bias_output,age_category) in horses.items():
        predicted_output = forward_pass(X, weights_input_hidden, weights_hidden_output,bias_hidden,bias_output)
        loss = mse_loss(y_one_hot, predicted_output)
        performances[horse_id]=loss


  sorted_performances = list(performances.items())
  n = len(sorted_performances)
  for i in range(n):
      for j in range(0, n-i-1):
          if sorted_performances[j][1] > sorted_performances[j+1][1]:
              sorted_performances[j], sorted_performances[j+1] = sorted_performances[j+1], sorted_performances[j]

  top_10_percent = sorted_performances[:int(len(sorted_performances) * (pN/100))]

  imitate_matrix_in=np.zeros_like(horses[0][0])
  imitate_matrix_out=np.zeros_like(horses[0][1])

  imitate_matrix_bias_in=np.zeros_like(horses[0][2])
  imitate_matrix_bias_out=np.zeros_like(horses[0][3])

  sum_weight_top10_in=np.zeros_like(horses[0][0])
  sum_weight_top10_out=np.zeros_like(horses[0][1])

  sum_weight_top10_bias_in=np.zeros_like(horses[0][2])
  sum_weight_top10_bias_out=np.zeros_like(horses[0][3])

  for horse_id, _ in top_10_percent:

    sum_weight_top10_in+=horses[horse_id][0]
    sum_weight_top10_out+=horses[horse_id][1]

    sum_weight_top10_bias_in+=horses[horse_id][2]
    sum_weight_top10_bias_out+=horses[horse_id][3]

  weights_input_hidden, weights_hidden_output,bias_hidden,bias_output, age_category=horses[id_horse]
  if age_category=='γ':
    imitate_matrix_in=i_gamma*((sum_weight_top10_in/pN)-horses[horse_id][0])
    imitate_matrix_out=i_gamma*((sum_weight_top10_out/pN)-horses[horse_id][1])

    imitate_matrix_bias_in=i_gamma*((sum_weight_top10_bias_in/pN)-horses[horse_id][2])
    imitate_matrix_bias_out=i_gamma*((sum_weight_top10_bias_out/pN)-horses[horse_id][3])

    i_gamma=i_gamma*wi
  return  imitate_matrix_in,imitate_matrix_out,imitate_matrix_bias_in,imitate_matrix_bias_out,i_gamma

In [46]:
def defense_mechanism(id_horse,horses,X,y_one_hot,d_alpha=0.5,d_beta=0.2,d_gamma=0.1,wd=0.95):
  qN=20
  performances={}
  for horse_id, (weights_input_hidden, weights_hidden_output, bias_hidden,bias_output,age_category) in horses.items():
        predicted_output = forward_pass(X, weights_input_hidden, weights_hidden_output,bias_hidden,bias_output)
        loss = mse_loss(y_one_hot, predicted_output)
        performances[horse_id]=loss


  sorted_performances = list(performances.items())
  n = len(sorted_performances)
  for i in range(n):
      for j in range(0, n-i-1):
          if sorted_performances[j][1] < sorted_performances[j+1][1]:
              sorted_performances[j], sorted_performances[j+1] = sorted_performances[j+1], sorted_performances[j]

  worst_20_percent = sorted_performances[:int(len(sorted_performances) * (qN/100))]

  defense_matrix_in=np.zeros_like(horses[0][0])
  defense_matrix_out=np.zeros_like(horses[0][1])

  defense_matrix_bias_in=np.zeros_like(horses[0][2])
  defense_matrix_bias_out=np.zeros_like(horses[0][3])

  sum_weight_worst20_in=np.zeros_like(horses[0][0])
  sum_weight_worst20_out=np.zeros_like(horses[0][1])

  sum_weight_worst20_bias_in=np.zeros_like(horses[0][2])
  sum_weight_worst20_bias_out=np.zeros_like(horses[0][3])

  for horse_id, _ in worst_20_percent:
    sum_weight_worst20_in+=horses[horse_id][0]
    sum_weight_worst20_out+=horses[horse_id][1]
    sum_weight_worst20_bias_in+=horses[horse_id][2]
    sum_weight_worst20_bias_out+=horses[horse_id][3]


  if age_category=='α':
    defense_matrix_in=-d_alpha*((sum_weight_worst20_in/qN)-horses[id_horse][0])
    defense_matrix_out=-d_alpha*((sum_weight_worst20_out/qN)-horses[id_horse][1])
    defense_matrix_bias_in=-d_alpha*((sum_weight_worst20_bias_in/qN)-horses[id_horse][2])
    defense_matrix_bias_out=-d_alpha*((sum_weight_worst20_bias_out/qN)-horses[id_horse][3])
    d_alpha=d_alpha*wd
  if age_category=='β':
    defense_matrix_in=-d_beta*((sum_weight_worst20_in/qN)-horses[id_horse][0])
    defense_matrix_out=-d_beta*((sum_weight_worst20_out/qN)-horses[id_horse][1])
    defense_matrix_bias_in=-d_beta*((sum_weight_worst20_bias_in/qN)-horses[id_horse][2])
    defense_matrix_bias_out=-d_beta*((sum_weight_worst20_bias_out/qN)-horses[id_horse][3])
    d_beta=d_beta*wd
  if age_category=='γ':
    defense_matrix_in=-d_gamma*((sum_weight_worst20_in/qN)-horses[id_horse][0])
    defense_matrix_out=-d_gamma*((sum_weight_worst20_out/qN)-horses[id_horse][1])
    defense_matrix_bias_in=-d_gamma*((sum_weight_worst20_bias_in/qN)-horses[id_horse][2])
    defense_matrix_bias_out=-d_gamma*((sum_weight_worst20_bias_out/qN)-horses[id_horse][3])
    d_gamma=d_gamma*wd
  return defense_matrix_in,defense_matrix_out,defense_matrix_bias_in,defense_matrix_bias_out,d_alpha,d_beta,d_gamma

In [47]:
def roam(weights,biases,age_category,r_delta=0.2,r_gamma=0.1,wr=0.95):
    random_matrix= np.random.rand(*weights.shape)
    roam_matrix= r_gamma * random_matrix * weights

    random_matrix_bias= np.random.rand(*biases.shape)
    roam_matrix_bias= r_gamma * random_matrix_bias * biases

    if age_category=='γ':
      r_gamma=r_gamma*wr
    if age_category=='δ':
      r_delta=r_delta*wr
    return roam_matrix,roam_matrix_bias,r_gamma,r_delta

In [48]:
g_factor_in=1.5
g_factor_out=1.5
g_factor_bias_in=1.5
g_factor_bias_out=1.5
h_beta=0.9
h_gamma=0.5
s_beta=0.2
s_gamma=0.1
i_gamma=0.3
r_gamma_in=0.05
r_gamma_out=0.05
r_delta_in=0.1
r_delta_out=0.1
d_alpha=0.5
d_beta=0.2
d_gamma=0.1
performances={}
for epoch in range(epochs):
    for horse_id, (weights_input_hidden, weights_hidden_output,bias_hidden,bias_output,age_category) in horses.items():
        #grazing
        grazing_matrix_in,g_factor_in=grazing(weights_input_hidden,g_factor=g_factor_in)
        grazing_matrix_out,g_factor_out=grazing(weights_hidden_output,g_factor=g_factor_out)
        grazing_matrix_bias_in,g_factor_bias_in=grazing(bias_hidden,g_factor=g_factor_bias_in)
        grazing_matrix_bias_out,g_factor_bias_out=grazing(bias_output,g_factor=g_factor_bias_out)

        #hierarchy
        hierarchy_matrix_in,hierarchy_matrix_out,hierarchy_matrix_bias_in,hierarchy_matrix_bias_out,h_beta,h_gamma=hierarchy(horse_id,horses,X,y_true_one_hot,h_beta=h_beta,h_gamma=h_gamma)
        #sociability
        sociability_matrix_in,sociability_matrix_out,sociability_matrix_bias_in,sociability_matrix_bias_out,s_beta,s_gamma=sociability(horse_id,horses,s_beta=s_beta,s_gamma=s_gamma)
        #imitate
        imitate_matrix_in,imitate_matrix_out,imitate_matrix_bias_in,imitate_matrix_bias_out,i_gamma=imitation(horse_id,horses,X,y_true_one_hot,i_gamma=i_gamma)
        #defense
        defense_matrix_in,defense_matrix_out,defense_matrix_bias_in,defense_matrix_bias_out,d_alpha,d_beta,d_gamma=defense_mechanism(horse_id,horses,X,y_true_one_hot,d_alpha=d_alpha,d_beta=d_beta,d_gamma=d_gamma)
        #roam
        roam_matrix_in,roam_matrix_bias_in,r_gamma_in,r_delta_in=roam(weights_input_hidden,bias_hidden,age_category,r_gamma=r_gamma_in,r_delta=r_delta_in)
        roam_matrix_out,roam_matrix_bias_out,r_gamma_bias_out,r_delta_out=roam(weights_hidden_output,bias_output,age_category,r_gamma=r_gamma_out,r_delta=r_delta_out)


        sum_gamma_in=grazing_matrix_in+defense_matrix_in+hierarchy_matrix_in+sociability_matrix_in+imitate_matrix_in+roam_matrix_in
        sum_gamma_out=grazing_matrix_out+defense_matrix_out+hierarchy_matrix_out+sociability_matrix_out+imitate_matrix_out+roam_matrix_out
        sum_delta_in=grazing_matrix_in+roam_matrix_in
        sum_delta_out=grazing_matrix_out+roam_matrix_out

        sum_gamma_bias_in=grazing_matrix_bias_in+defense_matrix_bias_in+hierarchy_matrix_bias_in+sociability_matrix_bias_in+imitate_matrix_bias_in+roam_matrix_bias_in
        sum_gamma_bias_out=grazing_matrix_bias_out+defense_matrix_bias_out+hierarchy_matrix_bias_out+sociability_matrix_bias_out+imitate_matrix_bias_out+roam_matrix_bias_out
        sum_delta_bias_in=grazing_matrix_bias_in+roam_matrix_bias_in
        sum_delta_bias_out=grazing_matrix_bias_out+roam_matrix_bias_out


        if age_category == 'α':

          current_weights_input_hidden, current_weights_hidden_output,current_bias_hidden,current_bias_output, current_age_category = horses[horse_id]

          updated_weights_input_hidden = current_weights_input_hidden + grazing_matrix_in + defense_matrix_in
          updated_weights_hidden_output = current_weights_hidden_output + grazing_matrix_out + defense_matrix_out

          updated_bias_hidden = current_bias_hidden + grazing_matrix_bias_in + defense_matrix_bias_in
          updated_bias_output = current_bias_output + grazing_matrix_bias_out + defense_matrix_bias_out

          horses[horse_id] = (updated_weights_input_hidden, updated_weights_hidden_output, updated_bias_hidden,updated_bias_output ,current_age_category)

        if age_category=='β':

          current_weights_input_hidden, current_weights_hidden_output,current_bias_hidden,current_bias_output, current_age_category = horses[horse_id]

          updated_weights_input_hidden = current_weights_input_hidden + grazing_matrix_in + defense_matrix_in + hierarchy_matrix_in + sociability_matrix_in
          updated_weights_hidden_output = current_weights_hidden_output + grazing_matrix_out + defense_matrix_out + hierarchy_matrix_out + sociability_matrix_out

          updated_bias_hidden = current_bias_hidden + grazing_matrix_bias_in + defense_matrix_bias_in + hierarchy_matrix_bias_in + sociability_matrix_bias_in
          updated_bias_output = current_bias_output + grazing_matrix_bias_out + defense_matrix_bias_out + hierarchy_matrix_bias_out + sociability_matrix_bias_out

          horses[horse_id] = (updated_weights_input_hidden, updated_weights_hidden_output, updated_bias_hidden,updated_bias_output ,current_age_category)

        if age_category=='γ':

          current_weights_input_hidden, current_weights_hidden_output,current_bias_hidden,current_bias_output, current_age_category = horses[horse_id]

          updated_weights_input_hidden = current_weights_input_hidden + sum_gamma_in
          updated_weights_hidden_output = current_weights_hidden_output + sum_gamma_out

          updated_bias_hidden = current_bias_hidden + sum_gamma_bias_in
          updated_bias_output = current_bias_output + sum_gamma_bias_out

          horses[horse_id] = (updated_weights_input_hidden, updated_weights_hidden_output, updated_bias_hidden,updated_bias_output ,current_age_category)
        if age_category=='δ':

          current_weights_input_hidden, current_weights_hidden_output,current_bias_hidden,current_bias_output, current_age_category = horses[horse_id]

          updated_weights_input_hidden = current_weights_input_hidden + sum_delta_in
          updated_weights_hidden_output = current_weights_hidden_output + sum_gamma_out

          updated_bias_hidden = current_bias_hidden + sum_delta_bias_in
          updated_bias_output = current_bias_output + sum_gamma_bias_out

          horses[horse_id] = (updated_weights_input_hidden, updated_weights_hidden_output, updated_bias_hidden,updated_bias_output ,current_age_category)




        predicted_output=forward_pass(X,horses[horse_id][0],horses[horse_id][1],horses[horse_id][2],horses[horse_id][3])
        loss = mse_loss(y_true_one_hot, predicted_output)
        predicted_labels = np.argmax(predicted_output, axis=1)
        true_labels = np.argmax(y_true_one_hot, axis=1)
        # Calcul Accuracy 
        accuracy = np.mean(predicted_labels == true_labels)
        performances[horse_id]=(loss,accuracy)
        
    if(epoch%2==0):
        print("epoche : ",epoch)
    if(epoch==epochs-1):
      break
    horses=update_age(horses,X,y_true_one_hot,input_size=4,hidden_size=hidden_size,output_size=output_size)

best_horse_id = None
best_performance = float('-inf')  # Initialize the best performance with an infinite negative value

for horse_id, (loss, accuracy) in performances.items():
  performance_metric = accuracy  # You can also use a combination of loss and precision
  if performance_metric > best_performance:
      best_performance = performance_metric
      best_horse_id = horse_id
print("Best Horse N°:",best_horse_id," ; accuracy",best_performance*100,"%")


epoche :  0
epoche :  2
epoche :  4
epoche :  6
epoche :  8
epoche :  10
epoche :  12
epoche :  14
epoche :  16
epoche :  18
epoche :  20
epoche :  22
epoche :  24
epoche :  26
epoche :  28
epoche :  30
epoche :  32
epoche :  34
epoche :  36
epoche :  38
epoche :  40
epoche :  42
epoche :  44
epoche :  46
epoche :  48
epoche :  50
epoche :  52
epoche :  54
epoche :  56
epoche :  58
epoche :  60
epoche :  62
epoche :  64
epoche :  66
epoche :  68
epoche :  70
epoche :  72
epoche :  74
epoche :  76
epoche :  78
epoche :  80
epoche :  82
epoche :  84
epoche :  86
epoche :  88
epoche :  90
epoche :  92
epoche :  94
epoche :  96
epoche :  98
Best Horse N°: 10  ; accuracy 75.0 %


In [77]:
# Test
predicted_output=forward_pass(X_test,horses[best_horse_id][0],horses[best_horse_id][1],horses[best_horse_id][2],horses[best_horse_id][3])
predicted_labels = np.argmax(predicted_output, axis=1)
test_labels = np.argmax(y_test_one_hot, axis=1)
# Calcul Accuracy
accuracy = np.mean(predicted_labels == test_labels)

# desplay the result of Test
print(Fore.LIGHTCYAN_EX + Style.DIM + "Test Accuracy: "+ Style.RESET_ALL,end='')
print(Fore.LIGHTRED_EX + Style.DIM + "{:.2f}%".format(accuracy*100) + Style.RESET_ALL)

Test Accuracy: 83.33%
